In [ ]:
Data Sources (NHANES Variables)

Dietary Data:

Total energy intake (kcal) 
Macronutrient distribution (carbohydrates, proteins, fats) 
Micronutrient intake (vitamins, minerals) 
Food group consumption (fruits, vegetables, dairy, meat, grains)


Health & Biomarkers:

BMI, waist circumference
Blood glucose, HbA1c (diabetes risk)
Cholesterol levels (LDL, HDL, triglycerides)
Blood pressure


Lifestyle & Demographics:

Physical activity levels
Smoking and alcohol consumption
Age, sex, ethnicity, socioeconomic status